In [1]:
import numpy as np
import numba

import emcee
import pystan

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

To show failure modes of ensemble sampling at high dimension, we will sample out of a multivariate standard Normal distribution (µ = 0 and σ = 1 for all variables). To use `emcee` to do the sampling, we need to code up the logarithm of the probability density function.

In [2]:
@numba.jit(nopython=True)
def lnprob(x):
    """Log PDF, where mu and sigma are both arrays."""
    return -len(x)/2 * np.log(2*np.pi) - np.sum(x**2)/2

## Sampling at low dimension

We will first use a low dimensional multivariate Gaussian with dimension 3.

Because of the simplicity of this distribution, we can sample out of it directly without using MCMC. We will call these our "true" samples. We'll grab a thousand of these.

In [7]:
n_dim = 3
true_samples = np.random.normal(0, 1, size=(1000, n_dim))

To make a comparison between these samples and those taken with ensemble sampling (and also HMC sampling using Stan), we will compute the log probability from the samples.

In [8]:
true_lnprob = np.array([lnprob(x) for x in true_samples])

With these in hand, we will now perform sampling using `emcee`. We have a dimensionality of three, so we will make sure we have plenty of walkers; we'll use 30.

In [9]:
n_walkers = 30   # number of MCMC walkers
n_burn = 1000    # "burn-in" period to let chains stabilize
n_steps = 1000   # number of MCMC steps to take after burn-in

# x0[i,j] is the starting point for walk i along variable j.
x0 = np.random.uniform(low=-0.1, high=0.1, size=(n_walkers, n_dim))

# Set up sampler
sampler = emcee.EnsembleSampler(n_walkers, n_dim, lnprob)

# Do burn-in
pos, prob, state = sampler.run_mcmc(x0, n_burn, storechain=False)

# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps, thin=25)

Now, let's compare the log probabilities we got from our true samples and those from `emcee`. We will plot them as ECDFs, so I will quickly write a little ECDF function.

In [10]:
def ecdf(data):
    return np.sort(data), np.arange(1, len(data) + 1) / len(data)

x_true, y_true = ecdf(true_lnprob)
x_emcee, y_emcee = ecdf(sampler.flatlnprobability)

p = bokeh.plotting.figure(width=400,
                          height=250,
                          x_axis_label='ln probability',
                          y_axis_label='ECDF')
p.circle(x_true, y_true, legend='true')
p.circle(x_emcee, y_emcee, legend='emcee', color='orange')
p.legend.location='top_left'
p.legend.click_policy = 'hide'

bokeh.io.show(p)

Both samplers give basically the same log probability, so there is no immediate problem with ensemble sampling.

## HMC at low dimension

As a check, let's generate samples using HMC via Stan. First, we'll build the Stan model.

In [12]:
model_code = """
data {
  int N;
}


parameters {
  real x[N];
}


model {
  x ~ normal(0.0, 1.0);
}
"""

sm = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9e1f3c0f49c6a9a01237bf624bdde281 NOW.
/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp_1brpomf/stanfit4anon_model_9e1f3c0f49c6a9a01237bf624bdde281_6583863508113526882.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Now we can specify the data and sample.

In [13]:
# Set up inputs to Stan sampler
data = dict(N=n_dim)

# Get samples
samples = sm.sampling(data=data, warmup=2000, iter=3000, chains=1)

# Compute log probability
stan_samples = samples.extract('x')['x']
stan_lnprob = np.array([lnprob(x) for x in stan_samples])

Now, let's add the log probability from these samples to our plot.

In [14]:
x_stan, y_stan = ecdf(stan_lnprob)
p.circle(x_stan, y_stan, legend='stan', color='red')

bokeh.io.show(p)

Indeed, the Stan samples overlap.

## Sampling at high dimension

Now, we will sample the 100-dimensional problem. We follow exactly the same procedure as we did for the 3-dimensional problem. First, we set up the problem and get our "true" samples.

In [15]:
n_dim = 1000

true_samples = np.random.normal(0, 1, size=(1000, n_dim))
true_lnprob = np.array([lnprob(x) for x in true_samples])

Next, we sample with `emcee`. Since we have 100 dimensions, we will use 1000 walkers.

In [21]:
n_walkers = 5000  # number of MCMC walkers
n_burn = 1000     # "burn-in" period to let chains stabilize
n_steps = 1000    # number of MCMC steps to take after burn-in

# x0[i,j] is the starting point for walk i along variable j.
x0 = np.random.uniform(low=-0.1, high=0.1, size=(n_walkers, n_dim))

# Set up sampler
sampler = emcee.EnsembleSampler(n_walkers, n_dim, lnprob)

# Do burn-in
pos, prob, state = sampler.run_mcmc(x0, n_burn, storechain=False)

# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps, thin=10)

And finally, we will sample with Stan.

In [22]:
# Set up inputs to Stan sampler
data = dict(N=n_dim)

# Get samples
samples = sm.sampling(data=data, warmup=1000, iter=2000, chains=1)

# Compute log probability
stan_samples = samples.extract('x')['x']
stan_lnprob = np.array([lnprob(x) for x in stan_samples])

We have a lot of samples from `emcee`, so just to make the plot swallow less memory, we will scramble the ln probability values for the ensemble sampling and plot only 1000 points.

In [23]:
x_true, y_true = ecdf(true_lnprob)
x_emcee, y_emcee = ecdf(np.random.choice(sampler.flatlnprobability, 1000))
x_stan, y_stan = ecdf(stan_lnprob)

p = bokeh.plotting.figure(width=400,
                          height=250,
                          x_axis_label='ln probability',
                          y_axis_label='ECDF')
p.circle(x_true, y_true, legend='true')
p.circle(x_emcee, y_emcee, legend='emcee', color='orange')
p.circle(x_stan, y_stan, legend='stan', color='red')
p.legend.location='top_left'
p.legend.click_policy = 'hide'

bokeh.io.show(p)